## Opencv & CNN & System video

In [1]:
import cv2
import requests
from io import BytesIO
# extract and plot each detected face in a photograph
from matplotlib import pyplot
from matplotlib.patches import Rectangle,Circle
# from mtcnn import MTCNN
# from mtcnn.mtcnn import MTCNN
# from google.colab.patches import cv2_imshow
import zipfile
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder,Normalizer
from sklearn.svm import SVC
from random import choice
from PIL import Image as Img
from numpy import savez_compressed,asarray,load,expand_dims
from keras.models import load_model
# from keras_facenet import FaceNet
import tensorflow as tf
from tensorflow.keras import layers, models
import time

In [2]:
# class Data_Using:

def unzip(zip_dir,out_dir):
    path_folder = out_dir

    zip_file_path = f'{zip_dir}'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(path_folder)


def load_dataset(directory):
  	# enumerate folders, on per class
  images = []
  path_d = []
  allpath = []
  y = []
  for subdir in os.listdir(directory):

    # print(y)
       	# enumerate files
    for filename in os.listdir(directory+subdir+'/'):
      path = directory + subdir + '/' + filename
          #labels
      y.append(subdir)
      path_d.append(path)
      # print(path_d)
  # allpath.extend(path_d)

  print(path_d)
	  	# store
    # images.append(path_d)
  return path_d , y


def load_data_image(path_image):
    img = cv2.imread(path_image)
    # img = cv2.cvtColor(img,cv2.COLOR)
    return img


def load_data_video(path_video):

    frame_list = []
    # Load the video content using OpenCV
    cap = cv2.VideoCapture(path_video)
    # Loop through the frames and display them
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Display the frame
        # cv2_imshow( frame)

        frame_list.append(frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()
    frame_list = np.array(frame_list)
    return frame_list



def load_data_system_camera():
    # define a video capture object
    vid = cv2.VideoCapture(0)
    while(True):
        # Capture the video frame
        ret, frame = vid.read()
        # Display the resulting frame
        cv2.imshow('frame', frame)
        # cv2.imwrite('video.mp4',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()



In [4]:
from functools import wraps
import sys
import io
def capture_output(func):
    """Wrapper to capture print output."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        try:
            return func(*args, **kwargs)
        finally:
            sys.stdout = old_stdout

    return wrapper

In [5]:
class Face_Detection_Recognition_Video:

    def __init__(self):
      # super.__init__(self)
      self.face_number = 0
      self.folder_number = 0
      # self.detector = MTCNN()
      # self.w_detect = capture_output( self.detector.detect_faces)

    # def preprocess(self,image):
    #   #normalize
    #     pixels = image.astype('float32')
    #     pixels /= 255.0
    #     return pixels

      # draw each face separately
    def opencv_img(self,img):
      # Load some pre-trained data on face frontal from opencv (haar cascade algorithm)
      # trained_face_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
      trained_face_data = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

      # Choose an image to detect faces in
      # img = cv2.imread(img)

      # Must convert to greyscale
      grayscaled_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # Detect Faces
      face_coordinates = trained_face_data.detectMultiScale(grayscaled_img)

      crop_face = []

      # Draw rectangles around the faces
      for (x, y, w, h) in face_coordinates:
          cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
          # img_crop.append(img[y:y + h, x:x + w])
          Face = img[y:y + h, x:x + w]
          Face = Img.fromarray(Face)
          Face = Face.convert('RGB')
          Face = Face.resize((160,160))
          Face = asarray(Face)#,dtype=np.float32)
          crop_face.append(Face)

      return crop_face





    def model_CNN(self):
      model_cnn = models.Sequential([
      layers.Conv2D(16, (3, 3), activation='relu', input_shape=(160, 160, 3)),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(32, (3, 3), activation='relu'),# input_shape=(160, 160, 3)),
      layers.MaxPooling2D((2, 2)),
      # layers.Dropout(0.2),
      layers.Conv2D(64, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      # layers.Dropout(0.2),
      layers.Conv2D(128, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Flatten(),
      layers.Dense(512, activation='relu'),
      layers.Dropout(0.2),  # Dropout for regularization
      layers.Dense(128, activation='relu'),
      # layers.Dropout(0.2),
      # layers.Dense(64, activation='relu'),
      layers.Dense(9, activation='softmax')  # num_classes is the number of identities
      ])

      return model_cnn

    def train_cnn_model(self,trainx,trainy,testx, testy,E_number):
      self.model = self.model_CNN()
      self.model.summary()
      self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
      self.model.fit(trainx, trainy, epochs= E_number)
      test_loss, test_acc = self.model.evaluate(testx, testy)
      return self.model , test_loss, test_acc


    def recognition_model(self,image,model):
      random_face = np.array(image)
      random_face = random_face.reshape(1,160,160,3)
      w_predict = capture_output(model.predict)
      pred = w_predict(random_face)
      # random_face_name = out_encoder.inverse_transform([random_face_class])
      out_encoder = LabelEncoder()
      out_encoder.fit(trainy)
      class_index = tf.argmax(pred, axis=1)
      # confidence = tf.reduce_max(pred[0][random_face_class])
      pred_name = out_encoder.inverse_transform(class_index)
      # print('Predicted: %s' % pred_name)
      # print('Expected: %s' % random_face_name[0])
      # print(f"Predicted class: {class_index}, Confidence: {confidence * 100:.2f}%")
      # pyplot.figure()
      # pyplot.imshow(random_face.reshape(160,160,3))
      # pyplot.show()
      return pred_name

    def save_faces(self,data_face):
      folder_name = 'Faces'+f'{self.folder_number}'
      if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
      cv2.imwrite(folder_name + '/' + f'{self.face_number}.jpg',data_face)
      self.face_number += 1

    def face_counting(self,):
      pass

In [ ]:
path = 'Facedata.zip'
output = '/Users/zahrad/Downloads/testvideo1/Facedata/photos'
data = unzip(path,output)

In [8]:
!rm -R /Users/zahrad/Downloads/testvideo1/Facedata/photos/__MACOSX
!find . -name "*.DS_Store" -type f -delete

rm: /Users/zahrad/Downloads/testvideo1/Facedata/photos/__MACOSX: No such file or directory


In [6]:
# Load dataset

def faces_dataset(directory):
  X = list()
  newlabels = list()
  files , labels = load_dataset(directory)
  print(len(files),len(labels))
  for fil,label in zip(files,labels):
    facedetect = Face_Detection_Recognition_Video()
    img = load_data_image(fil)
    if not type(img) == type(None):
      faces = facedetect.opencv_img(img)
    #  faces , pos = facedetect.get_face(img)

    if not len(faces)== 0:
      X.append(faces)
      newlabels.append(label)

  return asarray(X,dtype='object'), asarray(newlabels,dtype='object')



# paths train and validation
train_path = '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/'
val_path = '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/val/'
# load train dataset
trainX, trainy = faces_dataset(train_path)
# load test dataset
testX, testy = faces_dataset(val_path)
# save arrays to one file in compressed format
# savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

['/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/me.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_142814.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_143922.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210541.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210420.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210556.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_210609.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_140800.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20220609_212247.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/20230511_150134.jpg', '/Users/zahrad/Downloads/testvideo1/Facesdata/photos/train/zahra_dabiri/

In [7]:
# Prepare data for Cnn

Cnn_trainx = []
for p in trainX:
  Cnn_trainx.append(p[0])
Cnn_trainx = np.array(Cnn_trainx)
print(Cnn_trainx.shape)

Cnn_testx = []
for p in testX:
  Cnn_testx.append(p[0])
Cnn_testx = np.array(Cnn_testx)
print(Cnn_testx.shape)

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
train_y = out_encoder.transform(trainy)
print(train_y.shape)
test_y = out_encoder.transform(testy)
print(test_y.shape)

(231, 160, 160, 3)
(44, 160, 160, 3)
(231,)
(44,)


In [ ]:
# Train model
face_detect = Face_Detection_Recognition_Video()
train_model = face_detect.train_cnn_model(Cnn_trainx,train_y,Cnn_testx,test_y,100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 158, 158, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 79, 79, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 77, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 38, 38, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 36, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 18, 18, 64)        0

In [8]:
# Loads the weights
face_detect = Face_Detection_Recognition_Video()
new_model = tf.keras.models.load_model('/Users/zahrad/Downloads/Model/Mymodel.h5')

In [10]:
# start streaming video from webcam
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    start = time.time()
    faces = face_classifier.detectMultiScale(gray_image,scaleFactor=1.1, minNeighbors=8,minSize=(40, 40))
    stop = time.time()
    print('detect_time:',stop-start)
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
    return faces

video_capture = cv2.VideoCapture(0)

while True:

    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    faces = detect_bounding_box(video_frame)  # apply the function we created to the video frame

    cv2.imshow( label_html, video_frame)  # display the processed frame in a window named "My Face Detection Project"

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    for i in range(len(faces)):
        x1, y1, width, height = faces[i]
        x2, y2 = x1 + width, y1 + height   
        Face = video_frame[y1:y2, x1:x2]
        # resize pixels to the model size
        Face = Img.fromarray(Face)
        Face = Face.convert('RGB')
        Face = Face.resize((160,160))
        Face = asarray(Face)#,dtype=np.float32)
        stop_2= time.time()
        # print('extract_time:',stop_2-start_2)
        # crop_face.append(Face)
        start = time.time()
        recog_model = face_detect.recognition_model(Face,new_model)
        stop = time.time()
        # print('recog_time:',stop-start)
        txt = recog_model[0]
        # print(txt , type(txt))
        font = cv2.FONT_HERSHEY_SIMPLEX
        # fontScale
        fontScale = 1
        # Blue color in BGR
        color = (255, 0, 0)
        # Line thickness of 2 px
        thickness = 2
        cv2.putText(video_frame,txt,(x1,y1-10),font,fontScale,color,thickness)
        cv2.imshow( label_html, video_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


detect_time: 0.015169143676757812
detect_time: 0.0728299617767334
detect_time: 0.049063920974731445
detect_time: 0.0501711368560791
detect_time: 0.05020713806152344
detect_time: 0.048967838287353516
detect_time: 0.051679134368896484
detect_time: 0.051647186279296875
detect_time: 0.0494382381439209
detect_time: 0.05012202262878418
detect_time: 0.05103278160095215
detect_time: 0.04964399337768555
detect_time: 0.048449039459228516
detect_time: 0.04986095428466797
detect_time: 0.049680233001708984
detect_time: 0.04979991912841797
detect_time: 0.05006217956542969
detect_time: 0.04846382141113281
detect_time: 0.0488128662109375
detect_time: 0.048232078552246094
detect_time: 0.048078298568725586
detect_time: 0.04857015609741211
detect_time: 0.04868793487548828
detect_time: 0.04835700988769531
detect_time: 0.0492701530456543
detect_time: 0.04834628105163574
detect_time: 0.049677133560180664
detect_time: 0.05101728439331055
detect_time: 0.049410104751586914
detect_time: 0.049226999282836914
det

: 

In [ ]:

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    start = time.time()
    faces = face_classifier.detectMultiScale(gray_image,scaleFactor=1.1, minNeighbors=8,minSize=(40, 40))
    stop = time.time()
    print('detect_time:',stop-start)
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
    return faces
video_capture = cv2.VideoCapture(0)


while True:

    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    faces = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame

    cv2.imshow(
        "My Face Detection Project", video_frame
    )  # display the processed frame in a window named "My Face Detection Project"

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()